# Install pipelines SDK

#### Please wait till this cell completes and then run next cells. This just need to be run once per active kernel.

In [ ]:
%env RELEASE_VERSION=0.1.18
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade

# Import kfp pkgs

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [ ]:
client = kfp.Client()
client.list_experiments()

# Create a Dkube MNIST experiment

In [ ]:
mnist_experiment = client.create_experiment(name='Dkube - Mnist pl')

# Define e2e MNIST Pipeline with Dkube components

In [ ]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

dkube_training_op           = components.load_component_from_file("../components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("../components/serving/component.yaml")
dkube_viewer_op             = components.load_component_from_file('../components/viewer/component.yaml')

@dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist digits pipeline with dkube components'
)
def d3pipeline(
    #In notebook DKUBE_USER_ACCESS_TOKEN is automatically picked up from env variable
    auth_token  = os.getenv("DKUBE_USER_ACCESS_TOKEN"),
    #By default tf v1.14 image is used here, v1.13 or v1.14 can be used. 
    #Or any other custom image name can be supplied.
    #For custom private images, please input username/password
    training_container=json.dumps({'image':'docker.io/ocdr/d3-datascience-tf-cpu:v1.14', 'username':'', 'password': ''}),
    #Name of the workspace in dkube. Update accordingly if different name is used while creating a workspace in dkube.
    training_program="mnist",
    #Script to run inside the training container    
    training_script="python model.py",
    tuning = '{"parallelTrialCount":2,"maxTrialCount":4,"maxFailedTrialCount":2,"objective":{"type":"maximize","goal":0.99,"objectiveMetricName":"accuracy"},"algorithm":{"algorithmName":"random"},"parameters":[{"name":"--learning_rate","parameterType":"double","feasibleSpace":{"min":"0.01","max":"0.05"}},{"name":"--batch_size","parameterType":"int","feasibleSpace":{"min":"100","max":"200"}}]}',
    #Input datasets for training. Update accordingly if different name is used while creating dataset in dkube.    
    training_datasets=json.dumps(["mnist"]),
    #Input dataset mount paths
    training_input_dataset_mounts=json.dumps(["/opt/dkube/input"]),
    #Output models for training.
    training_outputs=json.dumps(["mnist"]),
    #Output dataset mount paths
    training_output_mounts=json.dumps(["/opt/dkube/output"]),
    #Request gpus as needed. Val 0 means no gpu, then training_container=docker.io/ocdr/dkube-datascience-tf-cpu:v1.12    
    training_gpus=0,
    #Any envs to be passed to the training program    
    training_envs=json.dumps([{"steps": 100}]),
    #Device to be used for serving - dkube mnist example trained on gpu needs gpu for serving else set this param to 'cpu'
    serving_device='cpu',
    serving_container=json.dumps({'image':'docker.io/ocdr/mnist-example-preprocess:2.0.4', 'username':'', 'password': ''})
    ):

    train       = dkube_training_op(auth_token, training_container,
                                    program=training_program, run_script=training_script,
                                    datasets=training_datasets, outputs=training_outputs,
                                    input_dataset_mounts=training_input_dataset_mounts,
                                    output_mounts=training_output_mounts,
                                    ngpus=training_gpus,
                                    envs=training_envs, tuning=tuning)
    serving     = dkube_serving_op(auth_token, train.outputs['artifact'], device=serving_device, serving_container=serving_container).after(train)
    inference   = dkube_viewer_op(auth_token, serving.outputs['servingurl'],
                                 'digits', viewtype='inference').after(serving)

# Compile and generate tar ball

In [ ]:
compiler.Compiler().compile(d3pipeline, 'dkube_mnist_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

# Create and Run pipeline

#### Click the dkube-training stage to see the enhanced Dkube Datascience dashboard, metrics and graphs. Click the dkube-viewer stage for the simple UI to test the model predecitions

In [ ]:
run = client.run_pipeline(mnist_experiment.id, 'mnist_classifier_pipeline', 'dkube_mnist_pl.tar.gz', params={})